In [620]:
# Import libraries
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('rfsoc_lib')
from rfsoc_calibration import RfsocCalibration
from rfsoc_pulses import RfsocPulses
from rfsoc_arb_pulses import RfsocArbPulses

import pickle
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [621]:
# Board libraries
from qick import *
import Pyro4

# Pyro communicates with board
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "192.168.2.99" # <-- SET IP ADDRESS
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)


QICK configuration:

	Board: RFSoC4x2

	Software version: 0.2.267
	Firmware timestamp: Thu Sep 28 15:12:10 2023

	Global clocks (MHz): tProcessor 307.200, RF reference 409.600

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (13.333 us)
		fs=4915.200 MHz, fabric=307.200 MHz, 32-bit DDS, range=4915.200 MHz
		DAC tile 0, blk 0 is DAC_B
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (13.333 us)
		fs=4915.200 MHz, fabric=307.200 MHz, 32-bit DDS, range=4915.200 MHz
		DAC tile 2, blk 0 is DAC_A

	2 readout channels:
	0:	axis_readout_v2 - configured by PYNQ
		fs=2457.600 MHz, decimated=307.200 MHz, 32-bit DDS, range=2457.600 MHz
		maxlen 16384 accumulated, 1024 decimated (3.333 us)
		triggered by output 7, pin 14, feedback to tProc input 0
		ADC tile 0, blk 0 is ADC_D
	1:	axis_readout_v2 - configured by PYNQ
		fs=2457.600 MHz, decimated=307.200 MHz, 32-bit DDS, range=2457.600 MHz
		maxlen 16384 accumulated, 1024 decimated (3.333 us)
		triggered by

In [652]:
# Import DAC and ssb calibrations
with open('pickles/calibrate_phase.pickle', 'rb') as handle:
    dac_phis = pickle.load(handle)

with open('pickles/ssb_params_30000.pickle', 'rb') as handle:
    ssb_params = pickle.load(handle)

with open('pickles/const_power.pickle', 'rb') as handle:
    const_power = pickle.load(handle)

calibration = RfsocCalibration(dac_phis,
                               ssb_params = ssb_params,
                            #    const_power = const_power,
                            #    gain_factor = 0.5,
                               )

In [653]:
raman_cooling = (37000, 1.0, 0.2, 0)
initialisation = (2000, 0)
pi_half_zero_phase = (11.5, 1.0, 0.2, 0)
wait_time = (8, 0)
pi_half_vary_phase = (11.5, 1.0, 0.2, 180)
readout = (2000, 0)

pi_half_seq_a = RfsocArbPulses(soccfg,
                               [(0.8, 0), pi_half_zero_phase, wait_time, pi_half_vary_phase],
                               calibration=calibration,
                               outsel='product',
                               ch_index=1
                               )
pi_half_seq_b = RfsocArbPulses(soccfg,
                               [pi_half_zero_phase, wait_time, pi_half_vary_phase],
                               calibration=calibration,
                               outsel='product',
                               ch_index=0
                               )

sequences = {'DAC_A': [raman_cooling, initialisation, pi_half_seq_a, readout],
             'DAC_B': [raman_cooling, initialisation, pi_half_seq_b, readout],
             'DIG_0': [(10, 1)]
             }

# delays = {"DAC_A": 0, "DAC_B": 0, "DIG_0": 38}
gains = {'DAC_A': 32766/2, 'DAC_B': 32766/2}

pulses = RfsocPulses(sequences,
                     calibration,
                    #  delays=delays,
                     gains=gains,
                     print_params=True
                     )


---------- DAC A ----------
CONFIG
	ch_type: DAC
	ch_index: 1
	delay: 0
	gain: 16383.0
	calibrated: True
	abs_gain: False
	num_pulses: 2
	duration: 41.031800000000004
PULSES
	lengths: [37.0, 0.0318]
	times: [0.0, 39.0]
	amps: [1.0, None]
	gains: [16383, 32766]
	freqs: [200.0, 200.0]
	phases: [0.0, None]
	styles: ['const', 'arb']
	outsels: [None, 'product']
CALIBRATION
	gains: [15782, 32566]
	phases: [0.0, 0.0]

---------- DAC B ----------
CONFIG
	ch_type: DAC
	ch_index: 0
	delay: 0
	gain: 16383.0
	calibrated: True
	abs_gain: False
	num_pulses: 2
	duration: 41.031
PULSES
	lengths: [37.0, 0.031]
	times: [0.0, 39.0]
	amps: [1.0, None]
	gains: [16383, 32766]
	freqs: [200.0, 200.0]
	phases: [0.0, None]
	styles: ['const', 'arb']
	outsels: [None, 'product']
CALIBRATION
	gains: [16383, 32566]
	phases: [217.5101, 0.0]

---------- DIG 0 ----------
CONFIG
	ch_type: DIG
	ch_index: 0
	delay: 0
	num_pulses: 1
	duration: 0.01
PULSES
	lengths: [0.01]
	times: [0.0]

END TIME: 41.031800000000004


In [672]:
prog = QickProgram(soccfg)
pulses.generate_asm(prog, reps=2)
# pulses.generate_asm(prog, reps=1)
pulses.config_internal_start(soc, prog)
soc.tproc.start() # Run program
print(prog)


// Program

        synci 200;
        regwi 0, $14, 1;
LOOP_I: regwi 1, $22, 174762667;                //freq = 174762667
        regwi 1, $23, 0;                        //phase = 0
        regwi 1, $25, 15782;                    //gain = 15782
        regwi 1, $26, 601190;                   //phrst| stdysel | mode | | outsel = 0b01001 | length = 11366 
        regwi 1, $27, 0;                        //t = 0
        set 1, 1, $22, $23, $0, $25, $26, $27;  //ch = 1, pulse @t = $27
        regwi 1, $22, 174762667;                //freq = 174762667
        regwi 1, $23, 0;                        //phase = 0
        regwi 1, $25, 32566;                    //gain = 32566
        regwi 1, $24, 0;                        //addr = 0
        regwi 1, $26, 524298;                   //phrst| stdysel | mode | | outsel = 0b01000 | length = 10 
        regwi 1, $27, 11981;                    //t = 11981
        set 1, 1, $22, $23, $24, $25, $26, $27; //ch = 1, pulse @t = $27
        regwi 0, $22, 1